# Beam Search

Useful resources:

* https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
* Taken from wikipedia (https://en.wikipedia.org/wiki/Beam_search):
    * In computer science, beam search is a heuristic search algorithm that explores a graph by expanding the most promising node in a limited set. Beam search is an optimization of best-first search that reduces its memory requirements. Best-first search is a graph search which orders all partial solutions (states) according to some heuristic. But in beam search, only a predetermined number of best partial solutions are kept as candidates. It is thus a greedy algorithm.


**Why do we need something like beam search?**

It is common for models developed for  natural language processing tasks such as caption generation, text summarization, and machine translation to output a probability distribution over each word in the vocabulary for each word in the output sequence. It is then left to a decoder process to transform the probabilities into a final sequence of words. Decoding the most likely output sequence involves searching through all the possible output sequences based on their likelihood. The size of the vocabulary is often tens or hundreds of thousands of words, or even millions of words. Therefore, the search problem is exponential in the length of the output sequence and is intractable (NP-complete) to search completely. Candidate sequences of words are scored based on their likelihood. It is common to use a greedy search or a beam search to locate candidate sequences of text.

## Greedy Search Decoder

In [7]:
import numpy as np
from math import log

# define a sequence of 10 words over a vocab of 5 words
data = [[0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1]]
data = np.array(data)

In [13]:
# greedy decoder only gives you one candidate
# beam search decoder gives you k candidates
def greedy_decoder(data):
    # index for largest probability each row
    # argmax: returns indices of the maximum values along an axis.
    return [np.argmax(s) for s in data]

In [9]:
# decode sequence
result = greedy_decoder(data)
print(result)

[4, 0, 4, 0, 4, 0, 4, 0, 4, 0]


## Beam Search Decoder

In [15]:
# beam search

'''
Issues with this code:

1. 

I’m not sure I understand why the example multiplies
log probabilities. Aren’t log probabilities normally 
added together to get the equivalent of multiplied 
real-value scalar probabilities?


0.5 * 0.5 * 0.25 = 0.0625

log(0.5) + log(0.5) + log(0.25) = -2.772588722239781
exp(-2.772588722239781) == 0.0624

whereas:
log(0.5) * log(0.5) * log(0.25) = -0.6660493039778589
exp(-0.6660493039778589) =~ 0.51374 != 0.625 ??

2.

Another issue with the code is the line
” for j in range(len(row))”

You’re iterating over all the data in row. 
In a typical text generation problem, there is a huge 
amount of words in a vocabulary, we probably don’t
want to iterate over all of them? Rather we would 
be interested in only top k probabilities.

3. 

Another problem is that log(1) = 0, so 
prod( log(p_i) ) ~ 0 for any sequence containing 
any character that is predicted with probability close to 1.0.

Ultimately, this leading to a pathological degeneracy of 
solutions during inference. In my results, it manifested 
itself as long runs of newline characters.

'''

def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
    return sequences

In [12]:
# Perform beam search and give me k=3 candidates
# with the highest likelihood
result = beam_search_decoder(data, 3)
# print result
for seq in result:
    print(seq)

[[4, 0, 4, 0, 4, 0, 4, 0, 4, 0], 0.025600863289563108]
[[4, 0, 4, 0, 4, 0, 4, 0, 4, 1], 0.03384250043584397]
[[4, 0, 4, 0, 4, 0, 4, 0, 3, 0], 0.03384250043584397]
